# 3. Lineare Gleichungssysteme

In [1]:
import numpy as np

Um Rundungsfehler deutlicher zu sehen, schreiben wir eine Hilfsfunktion, die auf $p$ signifikante Dezimalstellen rundet:

In [2]:
def signif(x, p):
    """Rundet x auf p signifikante Dezimalstellen."""
    x = np.asarray(x)
    x_positive = np.where(np.isfinite(x) & (x != 0), np.abs(x), 10**(p-1))
    mags = 10 ** (p - 1 - np.floor(np.log10(x_positive)))
    return np.round(x * mags) / mags

In [3]:
#Beispiel
print(signif(0.0012345, 3))
print(signif(-1234.5, 3))
print(signif(0, 3))

0.00123
-1230.0
0.0


## 3.1 Der Gauß-Algorithmus

In [4]:
def backsubst(U, y, p):
    """Rücksubstitution: Berechnet aus der rechten oberen Dreiecksmatrix U und rechten Seite y die Lösung x und rundet dabei auf p signifikante Dezimalstellen."""
    n = len(y)  #Indizes 0,1,...,n-1
    if not np.shape(U) == (n,n):
        raise Exception("Dimensionen von U und y passen nicht")
    x = np.zeros(n)
    for i in range(n-1, -1, -1):
        x[i] = y[i]
        for k in range(i+1, n):
            x[i] = signif(x[i] - signif(U[i,k]*x[k], p), p)
        x[i] = signif(x[i]/U[i,i], p)
    return np.array([x]).T  #Spaltenvektor

In [5]:
def forwsubst(L, b, p):
    """Vorwärtssubstitution: Berechnet aus der linken unteren Dreiecksmatrix L mit Einsen auf der Diagonale und der rechten Seite b die Zwischenlösung y."""
    n = len(b)  #Indizes 0,1,...,n-1
    if not np.shape(L) == (n,n):
        raise Exception("Dimensionen von L und b passen nicht")
    y = np.zeros(n)
    for i in range(1, n):
        y[i] = b[i]
        for j in range(0, i):
            y[i] = signif(y[i] - signif(L[i,j]*y[j], p), p)
        y[i] = signif(y[i]/L[i,i], p)
    return np.array([y]).T  #Spaltenvektor

### Beispiel zum Gauß'schen Eliminationsverfahren

In [6]:
A = np.array([[2.1, 2512, -2516], [-1.3, 8.8, -7.6], [0.9, -6.2, 4.6]])
b = np.array([[6.5], [-5.3], [2.9]])
Ab = np.concatenate((A, b), axis=1)
print("Erweiterte Koeffizientenmatrix Ab = \n {} \n".format(np.matrix(Ab)))
print()

x_exakt = np.array([[5], [1], [1]])
print("A*x_exakt - b = \n {} \n".format(np.matmul(A, x_exakt) - b))

Erweiterte Koeffizientenmatrix Ab = 
 [[ 2.100e+00  2.512e+03 -2.516e+03  6.500e+00]
 [-1.300e+00  8.800e+00 -7.600e+00 -5.300e+00]
 [ 9.000e-01 -6.200e+00  4.600e+00  2.900e+00]] 


A*x_exakt - b = 
 [[ 0.0000000e+00]
 [ 8.8817842e-16]
 [-4.4408921e-16]] 



Wir probieren verschiedene Pivotstrategien aus und runden dabei auf $p$ Dezimalstellen.

In [7]:
p = 5

### Spaltenmaximumstrategie
In diesem Fall identisch mit der Diagonalstrategie.

In [8]:
#Erster Eliminationsschritt
Ab1 = np.zeros((3,4))
Ab1[0,:] = Ab[0,:]
l10 = signif(Ab[1,0]/Ab[0,0], p)
l20 = signif(Ab[2,0]/Ab[0,0], p)
Ab1[1,1:] = signif(Ab[1,1:] - signif(l10*Ab[0,1:], p), p)
Ab1[2,1:] = signif(Ab[2,1:] - signif(l20*Ab[0,1:], p), p)
print("Ab1 = \n {} \n".format(np.matrix(Ab1)))

Ab1 = 
 [[ 2.1000e+00  2.5120e+03 -2.5160e+03  6.5000e+00]
 [ 0.0000e+00  1.5639e+03 -1.5651e+03 -1.2762e+00]
 [ 0.0000e+00 -1.0828e+03  1.0829e+03  1.1430e-01]] 



In [10]:
signif(l10*Ab[0,1:], p)

array([-1555.1   ,  1557.5   ,    -4.0238])

In [12]:
from math_so.utils import signif as mysignif
mysignif(l10*Ab[0,1:], p)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
#Zweiter Eliminationsschritt
Ab2 = np.zeros((3,4))
Ab2[[0,1],:] = Ab1[[0,1],:]
l21 = signif(Ab1[2,1]/Ab1[1,1], p)
Ab2[2,2:] = signif(Ab1[2,2:] - signif(l21*Ab1[1,2:], p), p)
print("Ab2 = \n {} \n".format(np.matrix(Ab2)))

In [ ]:
#Wir speichern die LU-Zerlegung für später
U = Ab2[0:3,0:3]
L = np.array([[1, 0, 0], [l10, 1, 0], [l20, l21, 1]])

#Rücksubstitution
x_spmax = backsubst(Ab2[:,0:3], Ab2[:,3], p)
print("x = \n {} \n".format(x_spmax))

### Relative Spaltenmaximumstrategie

In [ ]:
#Bestimme Quotienten aus Elementen der Spalte 0 und Betragssumme der jeweiligen Zeile
q = [abs(Ab[i,0])/np.sum(abs(Ab[i,0:3])) for i in range(0,3)]
print(q)

In [ ]:
#Vertausche Zeilen 
Ab1 = Ab[[2,1,0],:]
print("Ab1 = \n {} \n".format(Ab1))

In [ ]:
#Erster Eliminationsschritt
Ab2 = np.zeros((3,4))
Ab2[0,:] = Ab1[0,:]
l10 = signif(Ab1[1,0]/Ab1[0,0], p)
l20 = signif(Ab1[2,0]/Ab1[0,0], p)
Ab2[1,1:] = signif(Ab1[1,1:] - l10*Ab1[0,1:], p)
Ab2[2,1:] = signif(Ab1[2,1:] - l20*Ab1[0,1:], p)
print("Ab2 = \n {} \n".format(np.matrix(Ab2)))

In [ ]:
Ab1[1,1:] - l10*Ab1[0,1:]

In [ ]:
signif(Ab1[1,1:] - l10*Ab1[0,1:], p)

In [ ]:
#Bestimme Quotienten aus Elementen der Spalte 1 und Betragssumme der jeweiligen Zeile
q = [abs(Ab2[i,1])/np.sum(abs(Ab2[i,1:3])) for i in range(1,3)] 
print(q)

In [ ]:
#Vertausche Zeilen 
Ab3 = Ab2[[0,2,1], :]
print("Ab3 = \n {} \n".format(Ab3))

In [ ]:
#Zweiter Eliminationsschritt
Ab4 = np.zeros((3,4))
Ab4[[0,1],:] = Ab3[[0,1],:]
l21 = signif(Ab3[2,1]/Ab3[1,1], p)
Ab4[2,2:] = signif(Ab3[2,2:] - signif(l21*Ab3[1,2:], p), p)
print("Ab4 = \n {} \n".format(np.matrix(Ab4)))

In [ ]:
#Rücksubstitution
x_relspmax = backsubst(Ab4[:,0:3], Ab4[:,3], p)
print("x = \n {} \n".format(x_relspmax))

Deutlich genaueres Ergebnis als mit Spaltenmaximumstrategie!

### Nachiteration
Wir nehmen nochmal das Ergebnis der Spaltenmaximumstrategie und berechnen zunächst den Residuenvektor:

In [ ]:
print("x_spmax = \n {} \n". format(x_spmax))
r = np.matmul(A, x_spmax) - b
print("r = A*x_spmax - b = \n {} \n".format(r))

In [ ]:
#Löse Az = -r
y = forwsubst(L, -r, p)
z = backsubst(U, y, p)
x = x_spmax + z
print("x = \n {} \n".format(x))

Verbessertes Ergebnis!

### LU-Zerlegung mit scipy.linalg.lu

In [ ]:
from scipy.linalg import lu
Ps, Ls, Us = lu(A)
print("P mit scipy: \n {} \n".format(Ps))
print("L mit scipy: \n {} \n".format(Ls))
print("L gerundet mit Spaltenmaximumstrategie: \n {} \n".format(L))
print("U mit scipy: \n {} \n".format(Us))
print("U gerundet mit Spaltenmaximumstrategie: \n {} \n".format(U))